<a href="https://colab.research.google.com/github/Jaeji/AM360Paper/blob/main/Paper_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Section 1: ติดตั้งไลบรารีและระบบโหลดไฟล์ (Setup)

In [21]:
# ---------------------------------------------------------
# 1. IMPORT LIBRARIES (นำเข้าเครื่องมือที่จำเป็น)
# ---------------------------------------------------------
import os               # ใช้จัดการไฟล์และโฟลเดอร์ในเครื่อง
import shutil           # ใช้ย้ายไฟล์ (Move) หรือลบ (Delete)
import pandas as pd     # พระเอกสำหรับจัดการตารางข้อมูล
import numpy as np      # ใช้คำนวณเลขคณิต (หาค่าเฉลี่ย, ค่ามากสุด)
import xgboost as xgb   # สมอง AI ที่เราจะใช้เทรน (XGBoost)
import scipy.io         # จำเป็นมาก! ใช้เปิดไฟล์ .mat ของ NASA
from sklearn.model_selection import train_test_split # ใช้แบ่งข้อสอบ
from sklearn.metrics import mean_squared_error       # ใช้ตรวจคำตอบ

# =========================================================
# 🛠️ GITHUB CLONING PROCESS (กระบวนการดึงไฟล์)
# =========================================================
print("🚀 Initiating System Setup... (เริ่มการติดตั้งระบบ)")

# 1. ตั้งค่าชื่อ Repo และ URL ให้ตรงกับของคุณ
repo_url = "https://github.com/Jaeji/AM360Paper.git"
repo_name = "AM360Paper"  # ชื่อโฟลเดอร์ที่จะถูกสร้างตอน Clone

# 2. เคลียร์พื้นที่เก่า (เผื่อเคยรันแล้วมีโฟลเดอร์ค้างอยู่)
if os.path.exists(repo_name):
    shutil.rmtree(repo_name) # ลบทิ้งไปเลย จะได้เริ่มใหม่แบบสะอาดๆ

# 3. สั่ง Clone (ไปดูดข้อมูลมาจาก GitHub)
print(f"📡 Cloning from: {repo_url}...")
!git clone {repo_url}

# 4. ย้ายไฟล์ออกมาใช้งาน
source_path = f"{repo_name}/B0005.mat" # ระบุตำแหน่งไฟล์ที่ซ่อนอยู่
destination = "B0005.mat"              # ปลายทางที่จะเอามาวาง

if os.path.exists(source_path):
    # ย้ายไฟล์ออกมาไว้หน้าสุด (Current Directory)
    shutil.move(source_path, destination)
    print(f"✅ ย้ายไฟล์ {destination} เรียบร้อย!")

    # ลบโฟลเดอร์ต้นฉบับทิ้ง (Cleanup) เพราะได้ของแล้ว
    shutil.rmtree(repo_name)
    print("🧹 ลบโฟลเดอร์ขยะเรียบร้อย")

else:
    print(f"❌ Error: หาไฟล์ไม่เจอที่ {source_path}")

print("\n✅ Setup Complete: พร้อมรัน Section ถัดไป")

🚀 Initiating System Setup... (เริ่มการติดตั้งระบบ)
📡 Cloning from: https://github.com/Jaeji/AM360Paper.git...
Cloning into 'AM360Paper'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 12 (delta 2), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (12/12), 15.22 MiB | 34.02 MiB/s, done.
Resolving deltas: 100% (2/2), done.
✅ ย้ายไฟล์ B0005.mat เรียบร้อย!
🧹 ลบโฟลเดอร์ขยะเรียบร้อย

✅ Setup Complete: พร้อมรัน Section ถัดไป


# Section 2: ตัวจัดการข้อมูล (Data Loader)

In [22]:
class BatteryDataLoader:
    def load_nasa_data(self, filepath="B0005.mat"):
        print(f"📂 กำลังอ่านและแปลงข้อมูลจาก {filepath} ...")

        try:
            # ใช้ scipy เปิดไฟล์ MATLAB
            mat = scipy.io.loadmat(filepath)

            # เจาะเข้าไปในโครงสร้างไฟล์ (NASA ซ้อนกล่องมาหลายชั้นมาก)
            data = mat['B0005'][0][0][0][0]
            dataset = []

            # ตัวแปรฝากข้อมูล: เก็บค่าตอนชาร์จ ไว้ใช้คู่กับตอน Discharge
            last_charge_props = {'time_at_4_15v': 0, 'min_voltage': 0}

            # วนลูปอ่านข้อมูลทีละรอบ (Cycle)
            for i, cycle in enumerate(data):
                cycle_type = cycle['type'][0]

                # --- กรณีที่ 1: รอบชาร์จไฟ (Charge) ---
                if cycle_type == 'charge':
                    try:
                        d = cycle['data'][0][0]
                        t = d['Time'][0]             # เวลา
                        v = d['Voltage_measured'][0] # แรงดันไฟฟ้า

                        # หาช่วงเวลาที่แรงดันสูงเกิน 4.15V (ยิ่งแช่นาน แบตยิ่งเสื่อม)
                        high_v_idx = np.where(v >= 4.15)[0]
                        time_415 = t[high_v_idx[-1]] - t[high_v_idx[0]] if len(high_v_idx) > 1 else 0

                        # บันทึกค่าไว้
                        last_charge_props = {
                            'time_at_4_15v': time_415,
                            'min_voltage': np.min(v)
                        }
                    except: pass

                # --- กรณีที่ 2: รอบใช้ไฟ (Discharge) ---
                elif cycle_type == 'discharge':
                    try:
                        d = cycle['data'][0][0]
                        t = d['Time'][0]
                        v = d['Voltage_measured'][0]

                        # เก็บข้อมูลสำคัญลงตาราง
                        dataset.append({
                            'discharge_time': t[-1] - t[0],          # แบตอึดแค่ไหน (วินาที)
                            'max_voltage_discharge': np.max(v),      # แรงดันสูงสุดตอนเริ่มใช้
                            'min_voltage_charge': last_charge_props['min_voltage'], # ดึงค่าจากรอบชาร์จ
                            'time_at_4_15v': last_charge_props['time_at_4_15v'],    # ดึงค่าจากรอบชาร์จ
                            'Capacity': d['Capacity'][0][0]          # ความจุจริง (เอาไว้เฉลย)
                        })
                    except: pass

            # แปลงเป็นตาราง DataFrame
            df = pd.DataFrame(dataset)

            # --- สร้างคำตอบ (Target) สำหรับสอน AI ---
            # หาจุดที่แบตพัง (Capacity < 1.4)
            eol_idx = df[df['Capacity'] < 1.4].index
            eol = eol_idx[0] if len(eol_idx) > 0 else len(df)

            # คำนวณ RUL (อายุขัยที่เหลือ) = วันพัง - วันปัจจุบัน
            # ใช้เทคนิคแยกบรรทัดเพื่อแก้ Error .clip()
            df['RUL'] = eol - df.index
            df['RUL'] = df['RUL'].clip(lower=0) # ห้ามติดลบ

            print(f"✅ โหลดข้อมูลสำเร็จ: {len(df)} รอบการชาร์จ")
            return df

        except Exception as e:
            print(f"❌ Error ในการอ่านไฟล์: {e}")
            return None

# เรียกใช้งานทันที
loader = BatteryDataLoader()
df = loader.load_nasa_data()

📂 กำลังอ่านและแปลงข้อมูลจาก B0005.mat ...
✅ โหลดข้อมูลสำเร็จ: 168 รอบการชาร์จ


# Section 3: สร้างและเทรน AI (Model Training)

In [23]:
class RULEstimator:
    def __init__(self):
        # สร้างโมเดล AI (XGBoost Regressor)
        self.model = xgb.XGBRegressor(
            objective='reg:squarederror', # บอกว่าทำนายตัวเลข
            n_estimators=100,             # สร้างต้นไม้ตัดสินใจ 100 ต้น
            max_depth=5                   # ต้นไม้ลึกไม่เกิน 5 ชั้น (กันจำข้อสอบ)
        )
        # กำหนดชื่อคอลัมน์ที่จะใช้เป็นโจทย์ข้อสอบ
        self.features = ['discharge_time', 'max_voltage_discharge', 'min_voltage_charge', 'time_at_4_15v']

    def train(self, df):
        if df is None: return

        # เตรียมโจทย์ (X) และเฉลย (y)
        X = df[self.features]
        y = df['RUL']

        print("🤖 กำลังเทรนโมเดล AI...")
        self.model.fit(X, y) # สั่งสอน AI
        print("✅ AI เทรนเสร็จเรียบร้อย พร้อมใช้งาน!")

    def predict(self, df_row):
        # ฟังก์ชันทำนาย: รับข้อมูลเข้ามา -> ส่งค่า RUL กลับไป
        return self.model.predict(df_row[self.features])[0]

# สร้างและเทรนโมเดลทันที
ai_engine = RULEstimator()
ai_engine.train(df)

🤖 กำลังเทรนโมเดล AI...
✅ AI เทรนเสร็จเรียบร้อย พร้อมใช้งาน!


# Section 4: ระบบคิดเงิน (Pricing Strategy)

In [24]:
class PricingSystem:
    def __init__(self, base_price=20.0):
        self.base_price = base_price # กำหนดราคาเหมาจ่ายตั้งต้น ($20)

    def calculate_bill(self, ai_model, user_batt, station_batt):
        # 1. ให้ AI ตรวจสุขภาพแบต (ทำนาย RUL)
        rul_user = ai_model.predict(user_batt)      # แบตลูกค้า
        rul_station = ai_model.predict(station_batt) # แบตสถานี

        # 2. ตั้งราคาค่าเสื่อมสภาพ
        price_per_rul = 0.1 # ราคาต่อ 1 RUL

        # 3. คำนวณส่วนต่าง RUL
        # (ถ้าแบตสถานีอายุยืนกว่า -> ลูกค้าต้องจ่ายเพิ่ม)
        rul_diff = rul_station - rul_user
        cost_rul = rul_diff * price_per_rul

        # 4. คำนวณส่วนต่างคุณภาพไฟฟ้า (Voltage Quality)
        # (ถ้าแบตสถานีแรงดันดีกว่า -> คิดเงินเพิ่มอีก)
        volt_diff = (station_batt['max_voltage_discharge'].values[0] -
                     user_batt['max_voltage_discharge'].values[0]) * 2.0

        # 5. รวมยอดเงินสุทธิ
        total = self.base_price + cost_rul + volt_diff

        # ส่งใบเสร็จกลับไปเป็น Dictionary
        return {
            'RUL_User': rul_user,
            'RUL_Station': rul_station,
            'Base': self.base_price,
            'RUL_Cost': cost_rul,
            'Quality_Cost': volt_diff,
            'Total': max(0, total) # ยอดรวมห้ามติดลบ
        }

# สร้างระบบคิดเงินรอไว้
pricing_bot = PricingSystem()
print("✅ ระบบคิดเงินพร้อมทำงาน")

✅ ระบบคิดเงินพร้อมทำงาน


# Section 5: ทดสอบจริง (Main Simulation)

In [25]:
if df is not None:
    print("\n" + "="*45)
    print("🔋 เริ่มต้นสถานการณ์จำลอง: ลูกค้ามาขอเปลี่ยนแบต")
    print("="*45)

    # --- 1. เลือกข้อมูลตัวอย่างจากไฟล์จริง ---
    # แบตลูกค้า (สมมติว่าเป็นแบตเก่า ใกล้พัง) -> เอาข้อมูลแถวที่ 600
    user_batt_data = df.iloc[[600]] if len(df) > 600 else df.iloc[[-1]]

    # แบตสถานี (สมมติว่าเป็นแบตใหม่) -> เอาข้อมูลแถวที่ 5
    station_batt_data = df.iloc[[5]]

    # --- 2. คำนวณค่าบริการ ---
    bill = pricing_bot.calculate_bill(ai_engine, user_batt_data, station_batt_data)

    # --- 3. ปริ้นท์ใบเสร็จ ---
    print(f"ผลการตรวจสภาพแบตเตอรี่โดย AI:")
    print(f"   🔴 แบตเดิมของลูกค้า:   เหลืออายุใช้งาน {bill['RUL_User']:.1f} รอบ")
    print(f"   🟢 แบตใหม่ของสถานี:    เหลืออายุใช้งาน {bill['RUL_Station']:.1f} รอบ")
    print("-" * 45)
    print(f"🧾 รายละเอียดค่าบริการ:")
    print(f"   1. ค่าธรรมเนียมพื้นฐาน:    ${bill['Base']:.2f}")
    print(f"   2. ส่วนต่างอายุแบต (RUL): +${bill['RUL_Cost']:.2f}")
    print(f"   3. ส่วนต่างคุณภาพไฟ:      +${bill['Quality_Cost']:.2f}")
    print("=" * 45)
    print(f"💰 ยอดรวมที่ต้องชำระ:      ${bill['Total']:.2f}")
    print("=" * 45)

else:
    print("❌ ไม่สามารถรันได้ เนื่องจากโหลดข้อมูลไม่สำเร็จ")


🔋 เริ่มต้นสถานการณ์จำลอง: ลูกค้ามาขอเปลี่ยนแบต
ผลการตรวจสภาพแบตเตอรี่โดย AI:
   🔴 แบตเดิมของลูกค้า:   เหลืออายุใช้งาน 0.0 รอบ
   🟢 แบตใหม่ของสถานี:    เหลืออายุใช้งาน 119.0 รอบ
---------------------------------------------
🧾 รายละเอียดค่าบริการ:
   1. ค่าธรรมเนียมพื้นฐาน:    $20.00
   2. ส่วนต่างอายุแบต (RUL): +$11.90
   3. ส่วนต่างคุณภาพไฟ:      +$-0.03
💰 ยอดรวมที่ต้องชำระ:      $31.87
